## Conditions Data Download

This project allows you to specify a list of locations and data attributes to download in a CSV format from the AerisWeather [`conditions endpoint`](https://www.aerisweather.com/support/docs/api/reference/endpoints/conditions/). Your output will look something like this:

| &nbsp; | place.name | place.country | periods.tempF | periods.feelslikeF | periods.humidity |
| ------ | ------ | ------ | ------ | ------ | ------ |
| 0 | minneapolis | us | 79.71 | 79.71 | 83 |
| 1 | portland | us | 73.51 | 73.51 | 64 |
| 2 | idaho springs | us | 77.71 | 77.71 | 33 |
| 3 | berkley | gb | 71.13 | 71.13 | 62 |


### Getting Started

Before you begin, you will need to configure your AerisWeather client id / secret, locations, and data attributes (`request_fields`). You can omit the `request_fields` list which will return all data attributes.

In [17]:
import os

client_id = os.getenv('AERIS_CLIENT_ID')
client_secret = os.getenv('AERIS_CLIENT_SECRET')

request_fields = [
    'place.name',
    'place.country',
    'periods.tempF',
    'periods.feelslikeF',
    'periods.humidity',
]

location_list = [
    "fi",
    "helsinki, fi"
]

#### Fields

The `request_fields` list will follow the same dot notation outlined in the fields parameter section of our [Reducing Output](https://www.aerisweather.com/support/docs/api/getting-started/reducing-output/) documentation

#### Locations

You can provide any [location format that is supported](https://www.aerisweather.com/support/docs/api/reference/places/) by the AerisWeather API.

## Import Libraries

In [18]:
import json
from requests import request
from pathlib import Path
from typing import List

import pandas as pd

# sample url for current conditions
# http://api.aerisapi.com/conditions/55343?client_id=[CLIENT_ID]&client_secret=[CLIENT_SECRET]

## Aeris DataFrame and Looping Locations

The function below will be used to return a single [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) after passing a location and requested data attributes.

In [19]:
def aeris_api_dataframe(location: str, custom_fields: List[str] = None):
    formatted_fields = []
    if custom_fields is not None:
        formatted_fields = ','.join(custom_fields)

    print(f"retrieving data for {location}...")
    res = request(
        method="GET",
        url=f"https://api.aerisapi.com/conditions/{location}",
        params={
            "client_id": client_id,
            "client_secret": client_secret,
            "fields": formatted_fields,
        }
    )
    
    if res.status_code != 200:
        raise Exception(f"status code was not 200: {res.status_code}")
    
    api_response_body = json.loads(res.text)

    try:
        df_pre_period = pd.json_normalize(api_response_body['response'][0]).drop("periods", axis=1)
        df_periods = pd.json_normalize(api_response_body['response'][0], "periods", record_prefix="periods.")

        return df_pre_period.join(df_periods, how="cross")
    except IndexError:
        print(f"API Response did not contain periods. Verify request parameters are correct.\n\nRequest:\n{res.url}\n\nResponse:\n{api_response_body}")

def locations_loop(locations: List):
    all_locs = []

    for loc in locations:
        all_locs.append(aeris_api_dataframe(location=loc, custom_fields=request_fields))

    return pd.concat(all_locs, ignore_index=True)

## Completed DataFrame

With a single call to the `locations_loop` function, we can return a completed DataFrame with all locations:

In [20]:
full_df = locations_loop(locations=location_list)
display(full_df)

retrieving data for fi...
API Response did not contain periods. Verify request parameters are correct.

Request:
https://api.aerisapi.com/conditions/fi?client_id=WSU4BzfSK3N2oqOHAsoic&client_secret=Yhckg91ZEaHIn5wZwhhz5puG7cFjWQbZSHdZsSMZ&fields=place.name%2Cplace.country%2Cperiods.tempF%2Cperiods.feelslikeF%2Cperiods.humidity

Response:
{'success': False, 'error': {'code': 'invalid_location', 'description': 'The requested location was not found.'}, 'response': []}
retrieving data for helsinki, fi...


,place.name,place.country,periods.tempF,periods.feelslikeF,periods.humidity
0,helsinki,fi,34.48,34.48,81


## Next Steps

One common use case we see with conditions is to generate a CSV. Here's a simple way to generate a csv with the built-in [pandas csv tool](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html). After running the following section, your csv will be located in the `./csv_output` directory:

In [26]:
from datetime import datetime

now = datetime.now()
output_dir = Path('aeris_output')
output_dir.mkdir(exist_ok=True)

download_location = output_dir / f"conditions-download-{now.strftime('%Y-%m-%dT%H:%M')}.csv"
full_df.to_csv(download_location, encoding="utf-8")
print('csv created!')